In [1]:
import os
import datetime
import pytz

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

from scipy.signal import find_peaks

In [3]:
mpl.rcParams['figure.figsize'] = (10, 6)
mpl.rcParams['axes.grid'] = False
all_cols = ['Grid Feed In', 'External Energy Supply', 'Internal Power Supply', 'Current Power', 'Self Consumption', 'Ambient Temperature', 'Module Temperature', 'Total Irradiation']

In [4]:
df = pd.read_csv('https://itsci.mju.ac.th/downloads/watcharin/datasets/pv/d336000.csv.gz', compression='gzip')
row_count2 = df.shape[0]
print(f"Total rows: {row_count2}")
df.head(5)

Total rows: 5627613


,BUOY,TIMEOBS,LAT,LON,TEMP,DPTEMP,SLP,DIR,WSPD,WaterT,PER,HGT,SHIP
0,209,1577836140,-14.3,-170.5,99999.0,99999.0,999999.0,999,9999.0,28.5,10,1,NaN
1,4,1577835600,17.6,-152.4,24.5,20.6,1016.4,60,10.0,25.8,999,999,NaN
2,2,1577835600,17.0,-157.7,25.7,20.0,1015.7,70,10.0,26.6,999,999,NaN
3,44,1577836200,21.6,-58.6,25.4,18.7,1017.8,90,8.0,99999.0,7,1,NaN
4,43,1577836200,21.1,-64.8,26.3,21.6,999999.0,90,9.0,99999.0,8,2,NaN


In [5]:
#  Convert "TIMEOBS" column to datetime and add it as a new column
df['TIMEOBS_datetime'] = pd.to_datetime(df['TIMEOBS'], unit='s').dt.tz_localize('UTC')
df = df.sort_values(by='TIMEOBS_datetime', ascending=True)

# Display the first few rows to verify
df.head(5)

,BUOY,TIMEOBS,LAT,LON,TEMP,DPTEMP,SLP,DIR,WSPD,WaterT,PER,HGT,SHIP,TIMEOBS_datetime
383,103,1577718000,49.9,-2.9,9.5,6.7,1030.5,320,1.5,11.4,9,1,7KDT,2019-12-30 15:00:00+00:00
621,85,1577721600,28.7,-147.3,19.0,17.5,1028.6,120,9.3,20.0,3,1,H9LA,2019-12-30 16:00:00+00:00
434,23,1577728800,54.6,11.2,6.7,99999.0,1019.1,280,11.8,5.9,999,999,CDE18,2019-12-30 18:00:00+00:00
145,85,1577728800,45.7,-62.7,-1.2,-4.6,1024.1,140,5.7,99999.0,999,999,SHIP,2019-12-30 18:00:00+00:00
321,23,1577732400,54.6,11.2,6.7,3.5,1018.8,260,13.4,5.8,4,1,DBKV,2019-12-30 19:00:00+00:00


In [6]:
df2.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
BUOY,5627613.0,9.712879e+01,9.530667e+01,0.000000e+00,2.200000e+01,6.500000e+01,1.680000e+02,9.750000e+02
TIMEOBS,5627613.0,1.698885e+09,3.390617e+07,1.577718e+09,1.659416e+09,1.707500e+09,1.725612e+09,1.893456e+09
LAT,5627613.0,3.235397e+01,1.646356e+01,-9.100000e+01,2.450000e+01,3.500000e+01,4.310000e+01,9.850000e+01
LON,5627613.0,-8.971992e+01,6.672829e+01,-9.971000e+02,-1.245000e+02,-8.780000e+01,-7.510000e+01,8.201000e+02
TEMP,5627613.0,2.160493e+04,4.113700e+04,-7.060000e+01,1.370000e+01,2.370000e+01,2.940000e+01,9.999900e+04
DPTEMP,5627613.0,4.767463e+04,4.993759e+04,-8.120000e+01,1.480000e+01,2.630000e+01,9.999900e+04,9.999900e+04
SLP,5627613.0,3.005530e+05,4.577235e+05,1.000000e+02,1.013400e+03,1.018800e+03,9.999990e+05,9.999990e+05
DIR,5627613.0,3.685489e+02,3.608814e+02,0.000000e+00,1.000000e+02,2.200000e+02,3.600000e+02,9.990000e+02
WSPD,5627613.0,2.348243e+03,4.233151e+03,0.000000e+00,4.000000e+00,7.000000e+00,1.300000e+01,9.999000e+03
WaterT,5627613.0,1.406716e+04,3.474499e+04,-9.800000e+01,1.320000e+01,2.130000e+01,2.840000e+01,9.999900e+04


In [10]:
len(df2)

5627613

In [11]:
filtered_df = df[(df['LAT'] >= 17) & (df['LAT'] <= 19)]
filtered_df2 = filtered_df[(filtered_df['LON'] >= 99) & (filtered_df['LON'] <= 1)]
filtered_df2

,BUOY,TIMEOBS,LAT,LON,TEMP,DPTEMP,SLP,DIR,WSPD,WaterT,PER,HGT,SHIP,TIMEOBS_datetime
